In [1]:
import nltk
import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from multiprocessing import pool
from tqdm import tqdm_notebook as tqdm

apostrophe_dict = {

"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}
short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}
emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}


In [2]:
train_data = pd.read_csv("train_tweets.csv")
test_data = pd.read_csv("test_tweets.csv")


In [3]:
#DEL @USER
train_data['tweet'] = train_data.tweet.str.replace(r'\@user', ' ')
test_data['tweet'] = test_data.tweet.str.replace(r'\@user', ' ')

<ipython-input-3-16f6c2c6b3d6>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['tweet'] = train_data.tweet.str.replace(r'\@user', ' ')
<ipython-input-3-16f6c2c6b3d6>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['tweet'] = test_data.tweet.str.replace(r'\@user', ' ')


In [4]:
# TO LOWER
train_data['tweet'] = train_data.tweet.str.lower()
test_data['tweet'] = test_data.tweet.str.lower()

In [5]:
full_text_data = list(train_data.tweet.values) + list(test_data.tweet.values)
#CHANGE APOSTROPHE WORDs
#train_data_not_aposthophe = train_data.tweet.replace({r'(\s\w+\s?\'\s?\w+\s)' : r' '}, regex=True)
#test_data_not_aposthophe = test_data.tweet.replace({r'(\s\w+\s?\'\s?\w+\s)' : r' '}, regex=True)
import re
def change_words_from_dict(text_list, dict_words):
    for regexp in dict_words.keys():
        text_list = re.sub(re.escape(regexp), dict_words[regexp], text_list)

    return text_list
c_w = np.vectorize(change_words_from_dict)
train_data.tweet = c_w(train_data.tweet, apostrophe_dict)
test_data.tweet = c_w(test_data.tweet, apostrophe_dict)

In [6]:
#CHANGE SHORT WORD
train_data.tweet = c_w(train_data.tweet, short_word_dict)
test_data.tweet = c_w(test_data.tweet, short_word_dict)


In [7]:
#CHANGE EMOUTICON
train_data.tweet = c_w(train_data.tweet, emoticon_dict)
test_data.tweet = c_w(test_data.tweet, emoticon_dict)

In [8]:
#Пунктуация
train_data.tweet = train_data.tweet.replace({r'[^\w\s]' : r''}, regex=True)
test_data.tweet = test_data.tweet.replace({r'[^\w\s]' : r''}, regex=True)

In [9]:
#Спецсимволы
train_data.tweet = train_data.tweet.replace({r'[^a-zA-Z0-9]' : r' '}, regex=True)
test_data.tweet = test_data.tweet.replace({r'[^a-zA-Z0-9]' : r' '}, regex=True)


In [10]:
#Числа
train_data.tweet = train_data.tweet.replace({r'[^a-zA-Z]' : r' '}, regex=True)
test_data.tweet = test_data.tweet.replace({r'[^a-zA-Z]' : r' '}, regex=True)

In [11]:
def one_simbol(data):
    return ' '.join([w for w in data.split() if len(w)>1])
c_w = np.vectorize(one_simbol)

#один символ
train_data.tweet = c_w(train_data.tweet)
test_data.tweet = c_w(test_data.tweet)

In [12]:
def make_token_all_tweet(data):
    out = []
    for words in data:
        out.append(word_tokenize(words))
    return out

tweet_token=make_token_all_tweet(train_data.tweet)
train_data = train_data.assign(tweet_token=tweet_token)

tweet_token=make_token_all_tweet(test_data.tweet)
test_data = test_data.assign(tweet_token=tweet_token)

In [15]:
from nltk.corpus import stopwords

def make_filtered_data(data):
    out = []

    stopWords = set(stopwords.words('english'))
    for words in data:
        wordsFiltered = []
        for w in words:
            if w not in stopWords:
                wordsFiltered.append(w)
        out.append(wordsFiltered)
    return out

tweet_token_filtered = make_filtered_data(train_data.tweet_token)
train_data = train_data.assign(tweet_token_filtered=tweet_token_filtered)

tweet_token_filtered = make_filtered_data(test_data.tweet_token)
test_data = test_data.assign(tweet_token_filtered=tweet_token_filtered)

